<div style="background-color:#000;"><img src="pqn.png"></img></div>

## Imports and setup

We use these libraries to grab stock data from the web and handle math calculations. This gives us fast access to reliable information and powerful number-crunching tools.

In [ ]:
import numpy as np
import yfinance as yf

Here, we're setting up all the key variables for our analysis. We define what stock ticker to use, what type of option to price, and set all the basic financial stats.

In [ ]:
symbol = "PLTR"
option_type = "call"
risk_free_rate = 0.0425
time_to_expiry = 81 / 365
strike_price = 140

We'll use this setup every time we want to price an option. These values set the framework for all calculations that follow—we can easily swap in new values as needed.

## Pull and prepare market data

We're reaching out to the web to grab recent stock prices. This lets us see where the stock has been trading over the past 60 days and gives us the current price to use in our model.

In [ ]:
data = yf.download(symbol, period="60d", interval="1d", progress=False)
close_prices = data["Close"]
spot_price = close_prices.iat[-1, 0]
returns = np.log(close_prices.PLTR / close_prices.PLTR.shift(1)).dropna()
volatility = np.std(returns) * np.sqrt(252)

We grab the full set of historical closing prices, then pull the very latest price for our calculations. The returns are the day-to-day changes in price, and measuring how much those fluctuate tells us how volatile the stock has been. We adjust that measure to reflect a one-year timeframe, which aligns with standard practice for option pricing.

## Calculate Black-Scholes parameters

Here we're calculating important numbers used in the Black-Scholes option pricing model. These values help us figure out the expected risk and potential payout of the option.

In [ ]:
d1 = (np.log(spot_price / strike_price) + (risk_free_rate + 0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
d2 = d1 - volatility * np.sqrt(time_to_expiry)

By running these calculations, we set up the groundwork for fair pricing. We factor in how far the current stock price is from the strike, how interest rates nudge values, and how much the price bounces around. The outputs serve as building blocks for the final option price and sensitivity.

## Compute option price and Greeks

Now we calculate the fair price of the option, plus sensitivity measures that tell us how that price moves with the market. This gives us the full picture on both expected cost and risk.

In [ ]:
if option_type == "call":
    option_price = spot_price * np.exp(-0) * 0.5 * (1 + np.math.erf(d1 / np.sqrt(2))) - strike_price * np.exp(-risk_free_rate * time_to_expiry) * 0.5 * (1 + np.math.erf(d2 / np.sqrt(2)))
    delta = 0.5 * (1 + np.math.erf(d1 / np.sqrt(2)))
else:
    option_price = strike_price * np.exp(-risk_free_rate * time_to_expiry) * 0.5 * (1 - np.math.erf(d2 / np.sqrt(2))) - spot_price * np.exp(-0) * 0.5 * (1 - np.math.erf(d1 / np.sqrt(2)))
    delta = -0.5 * (1 - np.math.erf(d1 / np.sqrt(2)))

In [ ]:
gamma = np.exp(-d1 ** 2 / 2) / (spot_price * volatility * np.sqrt(2 * np.pi * time_to_expiry))

We decide if we're pricing a call or put, and then follow the path for the right formula. We get the expected option price, and also calculate how much the price could wiggle as the stock moves (Delta and Gamma). This gives us everything we need to judge whether an option is fairly priced or too risky to buy.

In [ ]:
option_price, delta, gamma

We finish by lining up our final answers: What would we pay for the option, and how would the price respond if the market moves? These results make it easy to weigh potential trades with real numbers in hand.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.